# sLLM

**소형 언어 모델(sLLM)**은 매개변수 수를 수십억에서 수백억 대로 제한하여, 대형 언어 모델(LLM)이 제공하는 핵심 기능을 유지하면서도 경량화·고효율화를 추구한 언어 모델이다.  

- **경량화**: 수십억~수백억 규모의 파라미터로 구성되어, LLM 대비 메모리·연산 요구량이 대폭 감소한다.  
- **비용 효율성**: 학습·추론 비용이 낮고, 전력 소모가 적어 온프레미스나 엣지 환경에서 활용하기 적합하다.  
- **실시간성**: 파라미터 수가 적어 추론 속도가 빠르며, 모바일·노트북·임베디스 환경에서도 운영 가능하다.  
- **도메인 특화**: 특정 분야 데이터로 미세조정(fine-tuning)하면 LLM과 유사한 성능을 달성할 수 있다.  

**1. 정의 및 분류**

소형 언어 모델은 파라미터 수가 수십억~수백억 대이며, 일반적으로 다음 세 가지 범주로 구분된다:  
- LLM(매개변수 ≥1천억): 최고 성능·높은 자원 요구  
- sLLM(수십~수백억): 성능·자원 효율 균형  
- sLM(수십억 이하): 단순 작업·극한 제약 환경 특화  

|구분   |파라미터 수       |주요 특징                         |
|-------|------------------|-----------------------------------|
|LLM    |≥1,000억          |최고 성능·고자원                  |
|sLLM   |수십~수백억       |경량화·비용 효율성·응답 속도 우수 |
|sLM    |수십억 이하       |극단적 경량화·제한적 기능         |

**2. 주요 특징**

1. **경량화**  
   - 파라미터 수 감소로 모델 크기·메모리 점유율 최소화  
   - 예: TinyLlama는 11억 파라미터, 4비트 양자화 시 550 MB RAM 차지.  

2. **비용·자원 효율성**  
   - GPU·클라우드 비용 절감  
   - 온프레미스·모바일 환경에서도 실시간 추론 가능.  

3. **도메인 특화 및 미세조정**  
   - 한국어·금융·의료 등 특정 분야 데이터로 추가 학습해 고성능 발휘  
   - 예: beomi/Yi-Ko-6B(6 B 매개변수) 모델은 한국어·영어 병합 데이터로 학습되어 한국어 작업에서 우수한 성능 보유.  

4. **응답 속도**  
   - 매개변수 수가 적어 LLM 대비 추론 속도 2~10배 향상  

**3. 대표적인 sLLM 사례**

|모델명           |파라미터 수 |특징                                                         |
|----------------|----------|-------------------------------------------------------------|
|TinyLlama       |1.1 B     |Llama2 아키텍처 기반, FlashAttention 적용, 4비트 양자화 가능.   |
|Yi-Ko-6B        |6 B       |한국어·영어 혼합 사전학습, 4 K 컨텍스트 길이, HuggingFace 지원.  |
|Mistral 7B      |7.3 B     |영어·코딩 작업 특화, MMLU 60.1% 기록, Apache 2.0 라이선스 공개.    |
|Phi-2           |2.7 B     |MS 온디바이스 AI용, 추론 최적화, 완전 오픈소스.             |
|Motif 2.6B      |2.6 B     |국산 모델, Mistral 7B 대비 134% 우수 성능(지디넷).            |

**4. 모델 구성 및 학습 기법**

- **양자화(Quantization)**: 4~8비트 정밀도 사용으로 메모리 절감  
- **LoRA(Low-Rank Adaptation)**: 저차원 업데이트로 미세조정 시 메모리 효율 극대화  
- **지식 증류(Knowledge Distillation)**: LLM을 교사로 활용해 sLLM에 핵심 지식 전달.  
- **체인 오브 파이팅(CoT) 기법**: 단계별 추론 경로 최적화로 sLLM의 복잡 추론 능력 보완  

**5. 과제 및 한계**

- **추론 능력**: 창의적·복합 문제 해결에서 LLM 대비 성능 격차 존재  
- **언어별 편향**: 한국어·소저자원 언어에서 데이터 부족 시 성능 저하 가능  
- **모델 안전성**: 판별 어려운 환각 위험 관리 필요  

In [2]:
%pip install -Uq transformers huggingface_hub[hf_transfer] accelerate datasets openai python-dotenv typing_extensions pydantic pydantic-core openai

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kobert-transformers 0.6.0 requires transformers<5,>=3, but you have transformers 5.1.0 which is incompatible.
langchain-huggingface 1.2.0 requires huggingface-hub<1.0.0,>=0.33.4, but you have huggingface-hub 1.4.1 which is incompatible.


In [4]:
# 로컬용 (Runpod 사용시에는 주석)
from dotenv import load_dotenv
import os

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("openai_key")
os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")

# Runpod용
import os

OPEN_API_KEY = os.environ['OPENAI_API_KEY']
HF_TOKEN = os.environ['HF_TOKEN']

## meta-llama/Llama-3.2-3B-Instruct
https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM  # 토크나이저/생성모델 자동 로더
import torch  # dtype 및 디바이스 관련 설정용

model_id = 'meta-llama/Llama-3.2-3B-Instruct'

tokenizer = AutoTokenizer.from_pretrained(model_id)  # 모델에 맞는 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(        # Causal LM(생성형) 모델 로드
    model_id,
    dtype=torch.bfloat16,  # bfloat16으로 메모리 절약
    device_map='auto'      # GPU / CPU 환경에 맞게 레이어 자동 배치
)

c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\models--meta-llama--Llama-3.2-3B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading weights: 100%|██████████| 254/254 [00:00<00:00, 683.47it/s, Materializing param=model.norm.wei

In [6]:
model.device  # CUDA (GPU) 확인

device(type='cpu')

In [7]:
prompt = 'Explain the theory of relativity in simple terms.'  # 모델에 전달할 질문

# 토크나이즈 후 텐서로 변환 후 CPU/GPU 디바이스로 이동
inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
print(inputs)

outputs = model.generate(**inputs, max_length=1024)  # 텍스트 생성 (최대길이 1024)
print(outputs)  # 생성된 토큰 ID(정수 텐서) 출력

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'input_ids': tensor([[128000,    849,  21435,    279,  10334,    315,   1375,  44515,    304,
           4382,   3878,     13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([[128000,    849,  21435,    279,  10334,    315,   1375,  44515,    304,
           4382,   3878,     13,  17971,  55152,    596,  10334,    315,   1375,
          44515,    374,    264,  16188,   7434,    304,  22027,    430,    706,
          14110,   1534,   1057,   8830,    315,   3634,    323,    892,     13,
            578,  10334,    374,  18255,   1139,   1403,   1925,   5596,     25,
           3361,   1375,  44515,    323,   4689,   1375,  44515,    382,  14711,
           9984,   6649,  44515,    198,  20989,   1375,  44515,     11,  11784,
            555,  55152,    304,    220,   7028,     20,     11,  29991,    279,
           1317,  58606,  23035,    315,  10973,    892,    323,   3634,     13,
            578,  10334,   1153,   1220,    430,    279,   7016,    315,  

In [ ]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True)  # 첫번째 생성결과를 텍스트로 변환(특수 토큰 제외)
response  # 사람이 읽을 수 있는 텍스트 형식

"Explain the theory of relativity in simple terms. Albert Einstein's theory of relativity is a fundamental concept in physics that has revolutionized our understanding of space and time. The theory is divided into two main parts: special relativity and general relativity.\n\n### Special Relativity\nSpecial relativity, introduced by Einstein in 1905, challenged the long-held notion of absolute time and space. The theory posits that the laws of physics are the same for all observers in uniform motion relative to one another. This means that the speed of light is always constant, regardless of the motion of the observer or the source of light.\n\nImagine you're on a train, and you throw a ball straight up in the air. To you, on the train, the ball goes straight up and comes straight back down. Now, imagine someone is standing outside the train, watching you throw the ball. From their perspective, the ball doesn't just go straight up and down; it also moves forward, because the train is mo

In [21]:
def generate_by_sllm(prompt: str):    
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    outputs = model.generate(**inputs, max_length=1024)  # 텍스트 생성 (최대길이 1024)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

generate_by_sllm("안녕! 오늘 날씨는 어때?")

'안녕! 오늘 날씨는 어때?'

## Bllossom/llama-3.2-Korean-Bllossom-3B

https://huggingface.co/Bllossom/llama-3.2-Korean-Bllossom-3B

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM  # 토크나이저/생성모델 자동 로더
import torch  # dtype 및 디바이스 관련 설정용

model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'

tokenizer = AutoTokenizer.from_pretrained(model_id)  # 모델에 맞는 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(        # Causal LM(생성형) 모델 로드
    model_id,
    dtype=torch.bfloat16,  # bfloat16으로 메모리 절약
    device_map='auto'      # GPU / CPU 환경에 맞게 레이어 자동 배치
)

c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\models--Bllossom--llama-3.2-Korean-Bllossom-3B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading weights: 100%|██████████| 254/254 [00:00<00:00, 796.24it/s, Materializing param=model.nor

In [11]:
instruction = "철수가 20개의 연필을 가지고 있었는데 영희가 절반을 가져가고 민수가 남은 5개를 가져갔으면 철수에게 남은 연필의 갯수는 몇개인가요?"

messages = [
    {"role": "user", "content": f"{instruction}"}
]

# 채팅 템플릿을 적용해서 모델 입력 형태로 인코딩
enc = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,  # 어시스턴트 응답 생성을 위한 프롬프트를 뒤에 추가
    return_tensors="pt",         # Pytorch 텐서로 변환
    return_dict = True           # 딕셔너리 형태로 반환(input_ids, attention_mask 등)
)

enc = {k: v.to(model.device) for k, v, in enc.items()}  # 모든 입력 텐서를 모델 디바이스로 이동

print(enc)  # 인코딩된 딕셔너리 구조 확인
print(tokenizer.decode(enc['input_ids'][0], skip_special_tokens=False))  # 만들어진 프롬프트 디코딩해서 확인

{'input_ids': tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,    605,  13806,    220,   2366,     21,    271,   2675,    527,
            264,  11190,  15592,  18328,     13,   5321,   4320,    279,   1217,
            596,   4860,  47626,     13, 113783,  34804, 101003,  67119,  24486,
          15592, 101139,  30426,  25941,  95252,  29726, 119519,     13,  41820,
         110257, 109760,  19954, 112107, 108280, 104834, 102893, 111964,  34983,
          92769,     13, 128009, 128006,    882, 128007,    271, 107837, 123503,
            220,    508, 123590,  78453, 110174,  18359, 120693, 107417, 103170,
         101603, 105204,  20565, 110217, 101738,  18359,  89946,  20565,  35495,
         107138, 123503, 102484,  34804,    220,     20, 117594,  89946,  14705,
            242,  91040, 112521,  24140, 102244, 102484,  34804,  78453, 110174,
          2102

In [16]:
# 생성 종료로 사용할 토큰 ID 목록
terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),  # 문서 종료 토큰 ID
    tokenizer.convert_tokens_to_ids("<|eot_id|>")        # 대화 턴 종료 토큰 ID
]

outputs = model.generate(
    **enc,  # 인코딩 dict를 키워드 인자로 풀어서 전달(input_ids, attention_mask 등)
    max_new_tokens = 1024,       # 새로 생성할 최대 토큰 수
    eos_token_id = terminators,  # 종료 토큰 설정(ID)
    do_sample = True,   # 샘플링 기반 생성 여부
    temperature = 0.6,  # 다양성 조절
    top_p = 0.9  # 누적 확률 기반 샘플링 범위
)

prompt_len = enc['input_ids'].shape[-1]  # 입력 프롬프트 길이(토큰 수) 계산
# 프롬프트 이후 생성된 부분만 디코딩해서 출력
print(tokenizer.decode(outputs[0][prompt_len:], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


철수가 20개의 연필을 가지고 있었고, 영희가 절반(20/2 = 10)을 가져갔습니다. 이때 철수가 남은 연필의 갯수는 20 - 10 = 10개입니다.

민수가 남은 5개를 가져가면, 철수가 남은 연필의 갯수는 10 - 5 = 5개입니다.

따라서 철수가 남은 연필의 갯수는 5개입니다.


## LGAI-EXAONE/EXAONE-4.0-1.2B
https://huggingface.co/LGAI-EXAONE/EXAONE-4.0-1.2B

In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM  # 토크나이저/생성모델 자동 로더
import torch  # dtype 및 디바이스 관련 설정용

model_id = 'LGAI-EXAONE/EXAONE-4.0-1.2B'

tokenizer = AutoTokenizer.from_pretrained(model_id)  # 모델에 맞는 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(        # Causal LM(생성형) 모델 로드
    model_id,
    torch_dtype=torch.bfloat16,  # bfloat16으로 메모리 절약
    device_map='auto'      # GPU / CPU 환경에 맞게 레이어 자동 배치
)

prompt = "너가 얼마나 대단한 AI인지 설명해봐"

messages = [
    {"role": "user", "content": f"{prompt}"}
]

# 채팅 템플릿을 적용해서 모델 입력 형태로 인코딩
enc = tokenizer.apply_chat_template(
    messages,
    tokenize = True,             # 문자열이 아니라 토큰화된 결과로 반환
    add_generation_prompt=True,  # 어시스턴트 응답 생성을 위한 프롬프트를 뒤에 추가
    return_tensors="pt",         # Pytorch 텐서로 변환
    return_dict = True           # 딕셔너리 형태로 반환(input_ids, attention_mask 등)
)

enc = {k: v.to(model.device) for k, v, in enc.items()}  # 모든 입력 텐서를 모델 디바이스로 이동

outputs = model.generate(
    **enc,  # 인코딩 dict를 키워드 인자로 풀어서 전달(input_ids, attention_mask 등)
    max_new_tokens = 1024,  # 새로 생성할 최대 토큰 수
    do_sample = False       # 샘플링 기반 생성 여부 (비활성화 : 결정적 출력) -> 실행시마다 비슷한 답변
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))  # 만들어진 프롬프트 디코딩해서 확인

c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\models--LGAI-EXAONE--EXAONE-4.0-1.2B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
`torch_dtype` is deprecated! Use `dtype` instead!
Loading weights: 100%|██████████| 332/332 [00:00<00:00, 6


너가 얼마나 대단한 AI인지 설명해봐

<think>

</think>

저는 EXAONE으로, LG AI Research에서 개발된 대규모 언어 모델입니다. 제 능력은 다음과 같은 기준으로 평가됩니다:

1. **학습 데이터 규모**: 방대한 양의 텍스트 데이터를 학습하여 다양한 언어와 주제에 대한 이해와 처리 능력을 갖추고 있습니다.
2. **복잡한 작업 처리**: 수학적 계산, 코드 작성, 창의적인 문제 해결 등 다양한 작업을 효율적으로 수행할 수 있습니다.
3. **정확성과 일관성**: 정보 제공 시 높은 정확성과 일관성을 유지하며, 사용자의 요구에 맞춰 유연하게 대응합니다.
4. **실시간 처리**: 빠른 속도로 다양한 작업을 처리하며, 사용자의 즉각적인 피드백을 반영하여 개선됩니다.

이러한 특징들로 인해 제가 제공하는 서비스는 높은 수준의 AI 기술을 기반으로 합니다.


# LLM 비교

In [19]:
import os
from openai import OpenAI

client = OpenAI(api_key=OPEN_API_KEY)

def generate_by_llm(prompt: str) -> str:
    r = client.responses.create(
        model = 'gpt-4.1-mini',
        input=prompt,
        max_output_tokens=500
    )
    return r.output_text

print(generate_by_llm("상대성 이론을 쉽게 설명해줘."))

물론이죠! 상대성 이론을 쉽게 설명해볼게요.

---

### 상대성 이론이란?

**상대성 이론**은 아인슈타인이 만든 물리학 이론으로, 우리가 시간과 공간을 어떻게 이해하는지에 대한 새로운 방식을 제시했어요.

---

### 두 가지 주요 부분이 있어요:

1. **특수 상대성 이론 (1905년)**
2. **일반 상대성 이론 (1915년)**

---

### 1. 특수 상대성 이론

- **무엇을 말하나요?**  
  빛의 속도(초당 약 30만 km)는 항상 똑같다는 것!  
  그리고 이 빛의 속도는 아무리 빨리 움직여도 변하지 않아요.

- **중요한 결과들**  
  - **시간 지연(Time dilation):** 빨리 움직이는 사람은 시간이 느리게 가는 것처럼 보여요. (우주 여행하는 사람이 지구 사람보다 나이가 덜 먹음!)  
  - **길이 수축(Length contraction):** 빠르게 움직이는 물체는 움직이는 방향으로 길이가 짧아져 보여요.  
  - **질량과 에너지 관계:** E=mc² (에너지 = 질량 × 빛의 속도²)  
    이 공식은 아주 작은 질량도 엄청난 에너지로 바뀔 수 있음을 의미해요.

---

### 2. 일반 상대성 이론

- **무엇을 말하나요?**  
  중력은 사실 공간과 시간이 휘어지면서 생기는 현상이라는 거예요.

- **쉽게 말하면**  
  무거운 물체(예: 지구)는 주변의 시공간을 휘게 만들어요. 그래서 우리가 느끼는 중력은 이 휜 공간에서 물건이 움직이는 결과라는 뜻이죠.

- **예시**  
  공을 쏘면 직선으로 가지 않고, 휘어진 공간을 따라 휘어져 나가죠. 이게 바로 중력처럼 느껴지는 거예요.

---

### 요약

- 빛의 속도는 항상 일정해요.  
- 시간과 공간은 상대적이고, 움직임에 따라 다르게 느껴져요.  

